In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
baseDir = os.getcwd()
print(baseDir)
print(os.listdir())
dataLocation = 'data/short_clean_closepx.csv'
dataFilePath = os.path.join(baseDir, dataLocation)

C:\Users\Gerardo Sandoval\Documents\GitHub\SPY_Underlying_Study
['.git', '.ipynb_checkpoints', 'data', 'my_helpers.ipynb', 'my_helpers.py', 'README.md', 'short_clean_closepx.csv', 'working_file.ipynb', 'working_file_corr.ipynb', 'working_file_corrResults.ipynb']


In [4]:
masterData = pd.read_csv(dataFilePath, index_col=0)

In [5]:
masterData.columns
masterData.head()

,MSFT,AAPL,AMZN,JPM,GOOG,GOOGL,XOM
Date,,,,,,,
2005-01-03,26.740000,4.520714,44.520000,39.150002,100.700043,101.456459,50.090000
2005-01-04,26.840000,4.567143,42.139999,38.410000,96.621567,97.347351,49.750000
2005-01-05,26.780001,4.607143,41.770000,38.490002,96.129768,96.851852,49.490002
2005-01-06,26.750000,4.610714,41.049999,38.709999,93.665794,94.369370,50.119999
2005-01-07,26.670000,4.946429,42.320000,38.400002,96.298668,97.022018,49.790001


<h2> Take the correlation test results and look at
    price movement via charts</h2>

In [6]:
import my_helpers

In [7]:
# Break masterData into new dictionary. The keys are the stock
# and the value is a dataframe where each column is a year

byStockAndYear = my_helpers.ByStockAndYear(masterData)
# drop 2019 year
for stock, data in byStockAndYear.items():
    del data[2019]

# Rolling Correlation of data
rollCorrByStockAndYear = {}
for stock in byStockAndYear:
    rollCorrByStockAndYear[stock] = byStockAndYear[stock].rolling(window=20)\
    .corr().dropna()

In [8]:
rollCorrByStockAndYear['MSFT'].head(10)

2005      2006      2007      2008      2009      2010      2011  \
20 2005  1.000000  0.292330 -0.608512  0.815974  0.790996  0.510769 -0.081692   
   2006  0.292330  1.000000  0.028904 -0.008812 -0.077044 -0.338333 -0.337276   
   2007 -0.608512  0.028904  1.000000 -0.527576 -0.483272 -0.188820  0.119591   
   2008  0.815974 -0.008812 -0.527576  1.000000  0.843301  0.758033 -0.043737   
   2009  0.790996 -0.077044 -0.483272  0.843301  1.000000  0.842707  0.058714   
   2010  0.510769 -0.338333 -0.188820  0.758033  0.842707  1.000000  0.274005   
   2011 -0.081692 -0.337276  0.119591 -0.043737  0.058714  0.274005  1.000000   
   2012 -0.816265 -0.025094  0.432575 -0.859393 -0.859893 -0.806080  0.030243   
   2013 -0.553592  0.210170  0.201919 -0.509269 -0.659325 -0.624615 -0.119493   
   2014 -0.092724  0.139333 -0.137315  0.070140 -0.062302 -0.026339 -0.255881   

             2012      2013      2014      2015      2016      2017      2018  
20 2005 -0.816265 -0.553592 -0.092724  0.187873  0.432200 -0.358442 -0.736829  
   2006 -0.025094  0.210170  0.139333 -0.713706  0.506081  0.523126  0.242809  
   2007  0.432575  0.201919 -0.137315 -0.148218 -0.489618  0.137117  0.439118  
   2008 -0.859393 -0.509269  0.070140  0.391096  0.299552 -0.569538 -0.866443  
   2009 -0.859893 -0.659325 -0.062302  0.463829  0.102078 -0.689567 -0.891701  
   2010 -0.806080 -0.624615 -0.026339  0.609635 -0.208036 -0.822081 -0.825821  
   2011  0.030243 -0.119493 -0.255881  0.113488 -0.726058  0.014469  0.090064  
   2012  1.000000  0.594816 -0.003375 -0.479227 -0.237303  0.649746  0.891994  
   2013  0.594816  1.000000  0.494569 -0.672499  0.195014  0.724992  0.699715  
   2014 -0.003375  0.494569  1.000000 -0.246431  0.329961  0.141533  0.059895

In [9]:
seasonResults_8 = my_helpers.SeasonCorrTest(rollCorrByStockAndYear,\
                                            dropNum=8, n=0.75)
seasonResults_9 = my_helpers.SeasonCorrTest(rollCorrByStockAndYear,\
                                            dropNum=9, n=0.75)

MSFT completed
AAPL completed
AMZN completed
JPM completed
GOOG completed
GOOGL completed
XOM completed
MSFT completed
AAPL completed
AMZN completed
JPM completed
GOOG completed
GOOGL completed
XOM completed


<h2>Cycle to SeasonCorrTest returns </h2>

In [ ]:
def ResultSlideShow(data):
    if type(data)==dict:
        for stock, value in data.items():
            print(stock)
            print()
            yield print(value)
    else:
        print('Wrong data type')

In [ ]:
next(ResultSlideShow(seasonResults_8))

<h2>Extract usefull information from correlation studies</h2>

In [10]:
seasonResults_8['MSFT'].index[0]

(2005, 249, 2006)

In [11]:
len(seasonResults_8['MSFT'])

120

In [13]:
# This cell provides me with the days that had a high level
# of correlation.
daysCorrelated_8 = {}
for stock in seasonResults_8:
    days = []
    for n in range(len(seasonResults_8[stock].index)):
        a, b, c = seasonResults_8[stock].index[n]
        days.append(b)
    days = list(set(days))
    days.sort()
    daysCorrelated_8[stock] = days

In [14]:
def JustDaysCorrelated(data):
    '''
    Input: returned item from func: SeasonCorrTest()
    Output: dictionary. Key=stock, value= pd.series of the days
        that had high correlation. High correlation was established in 
        SeasonCorrTest()
    '''
    request = {}
    for stock in data:
        days = []
        for n in range(len(data[stock].index)):
            a, b, c = data[stock].index[n]
            days.append(b)
        days = list(set(days))
        days.sort()
        request[stock] = days
    return request

In [15]:
JustDaysCorrelated(seasonResults_9)

{'MSFT': [],
 'AAPL': [24, 25, 26, 27, 30, 182],
 'AMZN': [138],
 'JPM': [64],
 'GOOG': [205, 206, 207, 208, 209, 210],
 'GOOGL': [205, 206, 207, 208, 209, 210],
 'XOM': [156, 202]}

In [113]:
forSaving = pd.Series(daysCorrelated)
forSaving.to_csv(path_or_buf='data\DaysWithCorrelation8.csv')

C:\ProgramData\Anaconda3\envs\myDataEnv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [88]:
'''
# Breaking down the multilevel index.
daysCorrelated = {}
for data in range(len(seasonResults_8['MSFT'].index)):
    a,b,c = seasonResults_8['MSFT'].index[data]
    print(f' {a}:{c} {b}')
'''

<h2> Looking at days correlated via charts</h2>

In [97]:
# Looking at the days that are correlated via a chart
data = byStockAndYear['MSFT'].iloc[20:60]
stock = byStockAndYear.keys
years = [2005, 2006, 2007, 2009, 2012, 2014, 2018]

fig, axes = plt.subplots(len(years),1)

for n in range(0,len(years)):
    axes[n].plot(byStockAndYear['MSFT'][years[n]].iloc[26:46])
plt.show()

    

<h2>Attempting to extract information from the data</h2>

In [18]:
# A dict of the days that had a correlation above the desired level
# Returns all the pairs created from the correlation matrix
daysCorrelatedResults = {}
for stock, data in daysCorrelated_8.items():
    dataB = []
    for day in data:
        end = byStockAndYear[stock].loc[day]
        start = byStockAndYear[stock].loc[(day-20)]
        pctChange = (end-start)/start
        #print(f'{stock} {day} : {round(pctChange.mean(),2)}')
        a, b = day, round(pctChange,2)
        dataB.append((a,b))
    daysCorrelatedResults[stock] = dataB

In [19]:
daysCorrelatedResults

{'MSFT': [(141, 2005    0.03
   2006    0.06
   2007    0.04
   2008   -0.05
   2009   -0.01
   2010    0.06
   2011    0.15
   2012   -0.02
   2013   -0.05
   2014    0.06
   2015    0.01
   2016    0.14
   2017    0.05
   2018    0.09
   dtype: float64), (142, 2005    0.02
   2006    0.07
   2007    0.00
   2008   -0.10
   2009   -0.01
   2010    0.08
   2011    0.11
   2012   -0.04
   2013   -0.09
   2014    0.07
   2015    0.00
   2016    0.17
   2017    0.07
   2018    0.12
   dtype: float64), (201, 2005   -0.05
   2006    0.05
   2007    0.10
   2008   -0.04
   2009    0.04
   2010   -0.00
   2011    0.01
   2012   -0.05
   2013    0.04
   2014   -0.08
   2015    0.08
   2016    0.01
   2017    0.04
   2018   -0.01
   dtype: float64), (202, 2005   -0.02
   2006    0.05
   2007    0.05
   2008   -0.05
   2009    0.02
   2010    0.03
   2011    0.04
   2012   -0.06
   2013    0.07
   2014   -0.06
   2015    0.09
   2016   -0.00
   2017    0.05
   2018   -0.04
   dtype: float64), (2

In [21]:
# lets pull some price return out based on the days that are correlated
daysCorrelatedResultsSummary = {}
for stock, data in daysCorrelated_8.items():
    dataB = []
    for day in data:
        end = byStockAndYear[stock].loc[day]
        start = byStockAndYear[stock].loc[(day-20)]
        pctChange = (end-start)/start
        #print(f'{stock} {day} : {round(pctChange.mean(),2)}')
        a, b = day, round(pctChange.mean(),2)
        dataB.append((a,b))
    daysCorrelatedResultsSummary[stock] = dataB
        

In [22]:
daysCorrelatedResultsSummary

{'MSFT': [(141, 0.04),
  (142, 0.03),
  (201, 0.01),
  (202, 0.01),
  (203, 0.02),
  (204, 0.02),
  (205, 0.02),
  (214, 0.06),
  (216, 0.08),
  (217, 0.07),
  (249, 0.02),
  (250, 0.01)],
 'AAPL': [(23, -0.01),
  (24, -0.01),
  (25, -0.02),
  (26, -0.02),
  (27, -0.02),
  (28, -0.01),
  (29, -0.01),
  (30, 0.02),
  (60, 0.04),
  (61, 0.04),
  (181, 0.05),
  (182, 0.05),
  (183, 0.04),
  (184, 0.04),
  (185, 0.04),
  (189, 0.02)],
 'AMZN': [(27, -0.01),
  (29, 0.0),
  (90, 0.07),
  (91, 0.07),
  (92, 0.06),
  (137, 0.04),
  (138, 0.04),
  (139, 0.04),
  (140, 0.04),
  (214, 0.03)],
 'JPM': [(22, -0.0),
  (63, 0.06),
  (64, 0.08),
  (65, 0.07),
  (66, 0.05),
  (67, 0.04),
  (103, -0.02),
  (105, -0.03),
  (106, -0.04)],
 'GOOG': [(147, 0.04),
  (181, 0.04),
  (201, 0.03),
  (202, 0.03),
  (204, 0.05),
  (205, 0.06),
  (206, 0.06),
  (207, 0.06),
  (208, 0.07),
  (209, 0.07),
  (210, 0.07)],
 'GOOGL': [(147, 0.04),
  (148, 0.04),
  (181, 0.04),
  (201, 0.03),
  (202, 0.03),
  (205, 0.06)

In [23]:
# This iterates through the results, only printing the values
# where the return was greater than 5%.
for stock, data in daysCorrelatedResultsSummary.items():
    print(stock)
    for value in data:
        if value[1] >= 0.05:
            print(value)

MSFT
(214, 0.06)
(216, 0.08)
(217, 0.07)
AAPL
(181, 0.05)
(182, 0.05)
AMZN
(90, 0.07)
(91, 0.07)
(92, 0.06)
JPM
(63, 0.06)
(64, 0.08)
(65, 0.07)
(66, 0.05)
GOOG
(204, 0.05)
(205, 0.06)
(206, 0.06)
(207, 0.06)
(208, 0.07)
(209, 0.07)
(210, 0.07)
GOOGL
(205, 0.06)
(206, 0.06)
(207, 0.06)
(208, 0.06)
(209, 0.07)
(210, 0.07)
XOM


In [24]:
# This iterates through the results, only printing the values
# where the return was greater than 5%.
for stock, data in daysCorrelatedResultsSummary.items():
    print(stock)
    for value in data:
        if value[1] >= 0.04:
            print(value)

MSFT
(141, 0.04)
(214, 0.06)
(216, 0.08)
(217, 0.07)
AAPL
(60, 0.04)
(61, 0.04)
(181, 0.05)
(182, 0.05)
(183, 0.04)
(184, 0.04)
(185, 0.04)
AMZN
(90, 0.07)
(91, 0.07)
(92, 0.06)
(137, 0.04)
(138, 0.04)
(139, 0.04)
(140, 0.04)
JPM
(63, 0.06)
(64, 0.08)
(65, 0.07)
(66, 0.05)
(67, 0.04)
GOOG
(147, 0.04)
(181, 0.04)
(204, 0.05)
(205, 0.06)
(206, 0.06)
(207, 0.06)
(208, 0.07)
(209, 0.07)
(210, 0.07)
GOOGL
(147, 0.04)
(148, 0.04)
(181, 0.04)
(205, 0.06)
(206, 0.06)
(207, 0.06)
(208, 0.06)
(209, 0.07)
(210, 0.07)
XOM


In [26]:
resultsSeasonalCorrelation = {}
for stock, day in daysCorrelated_8.items():
    print(f'{stock}: {day}')

MSFT: [141, 142, 201, 202, 203, 204, 205, 214, 216, 217, 249, 250]
AAPL: [23, 24, 25, 26, 27, 28, 29, 30, 60, 61, 181, 182, 183, 184, 185, 189]
AMZN: [27, 29, 90, 91, 92, 137, 138, 139, 140, 214]
JPM: [22, 63, 64, 65, 66, 67, 103, 105, 106]
GOOG: [147, 181, 201, 202, 204, 205, 206, 207, 208, 209, 210]
GOOGL: [147, 148, 181, 201, 202, 205, 206, 207, 208, 209, 210]
XOM: [77, 79, 80, 97, 153, 154, 156, 157, 180, 181, 182, 183, 184, 185, 201, 202, 203, 206, 207, 208, 209, 210, 211]


In [ ]:
"""
I am leaving off analysing the results of the correlations study.
Next steps might be to 
1. run machine learning regression and seasonality
2. check if its possible to test seasonality with out correlation
3. start making notebooks presentable
"""